# Test performance

In [1]:
import numpy as np
import random
from tqdm import tqdm

## Create resources

In [2]:
tag_words = ['luck','nursery','stove','build','admire','store','reduction','meal','paper','general','tighten','summer','winter','gift','and','paste','conscience','neat','continue','apple','modest','respect','voice','bicycle','fancy','strict','tighten','hurt','slavery','bottle','fry','adopt','relation','bind','disappearance','fruit','cultivation','tour','bathe','nest','tend','laugh','dip','disappoint','ball','hasten','give','proof','flesh','whatever']

In [3]:
import sys
sys.path.append("../backend")

In [4]:
from rtm import Resource, Manager

In [ ]:
def create_tags(probs: list[float], total: int, dict: list[str]) -> list[str]:
    tag_list = []
    i = 0
    def generate_next(ps: list[float], parent_tag: str=None):
        nonlocal i
        if len(ps)>1:
            prob, n_ps = ps[0], ps[1:]
        else:
            prob, n_ps = ps[0], ps[:]
        
        while np.random.random()<prob and i<total:
            current_tag = random.choice(dict)
            current_full = f"{parent_tag}:{current_tag}" if parent_tag is not None else current_tag
            tag_list.append(current_full)
            i += 1
            generate_next(n_ps, current_full)

    while i<total:
        generate_next(probs)
    
    return tag_list

In [6]:
tags = create_tags([0.2, 0.8, 0.4], 30, tag_words)
tags

['cultivation',
 'cultivation:ball',
 'cultivation:apple',
 'conscience',
 'conscience:fry',
 'conscience:fry:tend',
 'conscience:nest',
 'build',
 'build:disappoint',
 'build:disappoint:tighten',
 'build:disappoint:tighten:disappearance',
 'build:disappoint:fancy',
 'build:disappoint:fancy:tend',
 'build:disappoint:fancy:tend:conscience',
 'build:disappoint:fancy:tend:conscience:relation',
 'build:disappoint:fancy:tend:conscience:winter',
 'build:disappoint:fancy:tend:conscience:winter:and',
 'build:disappoint:disappoint',
 'build:bottle',
 'build:luck',
 'build:nursery',
 'build:disappearance',
 'disappearance',
 'proof',
 'proof:fruit',
 'proof:paste',
 'proof:paste:nest',
 'proof:paste:cultivation',
 'proof:paste:and',
 'proof:respect']

In [7]:
def create_resources(avg_no_tags: int, total: int, tags: list[str]):
    resources = []
    for i in range(total):
        n = max(int(np.random.normal(loc=avg_no_tags, scale=2.5)), 1)
        res_tags = random.choices(tags, k=n)
        resources.append(Resource(f"Res {i+1}", ";;".join(res_tags)))
    return resources

In [8]:
resources = create_resources(3, 30, tags)
for r in resources:
    print(r.tags)

['build:luck', 'proof:respect']
['build:disappoint:fancy:tend:conscience:relation']
['cultivation:apple']
['build:disappoint:fancy:tend', 'cultivation', 'cultivation:ball', 'build:disappoint', 'conscience']
['disappearance', 'proof:paste']
['proof:paste:cultivation']
['build:disappearance']
['conscience', 'build:disappoint:fancy:tend:conscience:winter:and', 'build:disappoint:tighten:disappearance']
['proof:paste:cultivation', 'build', 'build:disappoint:fancy', 'build:luck', 'conscience:fry']
['build:disappoint:fancy:tend:conscience']
['cultivation:apple', 'build:disappoint:fancy', 'proof:paste:nest', 'build:luck', 'build']
['proof:paste:cultivation']
['build:disappoint:fancy', 'proof:fruit']
['proof:paste', 'build:disappoint:fancy:tend:conscience:relation', 'build:disappoint:fancy:tend:conscience:winter:and']
['proof:paste:nest', 'conscience:fry', 'cultivation:apple', 'proof:fruit', 'proof:paste:and', 'proof:paste']
['proof:fruit', 'build:disappoint:fancy:tend:conscience:relation', 'pr

## Memory resource test

In [9]:
manager1 = Manager()

In [10]:
for i in tqdm(range(1_000_000)):
    n = max(int(np.random.normal(loc=3, scale=2.5)), 1)
    res_tags = random.choices(tags, k=n)
    res = Resource(f"Res {i+1}", ";;".join(res_tags))
    manager1.add_resource(res)

100%|██████████| 1000000/1000000 [00:08<00:00, 123774.08it/s]


In [12]:
manager1.filter_resources(['disappearance', 'build:disappoint:disappoint', 'proof:paste:cultivation'])

[Resource<Res 551633>,
 Resource<Res 581043>,
 Resource<Res 815473>,
 Resource<Res 558093>,
 Resource<Res 701403>,
 Resource<Res 17662>,
 Resource<Res 89742>,
 Resource<Res 210102>,
 Resource<Res 783003>,
 Resource<Res 436882>,
 Resource<Res 740672>,
 Resource<Res 708031>,
 Resource<Res 189702>,
 Resource<Res 125098>,
 Resource<Res 228975>,
 Resource<Res 181205>,
 Resource<Res 923084>,
 Resource<Res 313806>,
 Resource<Res 751555>,
 Resource<Res 235946>,
 Resource<Res 510156>,
 Resource<Res 906595>,
 Resource<Res 305817>,
 Resource<Res 848456>,
 Resource<Res 961171>,
 Resource<Res 598557>,
 Resource<Res 183075>,
 Resource<Res 799159>,
 Resource<Res 345769>,
 Resource<Res 425669>,
 Resource<Res 673019>,
 Resource<Res 963044>,
 Resource<Res 283040>,
 Resource<Res 900311>,
 Resource<Res 561161>,
 Resource<Res 777403>,
 Resource<Res 932272>,
 Resource<Res 514068>,
 Resource<Res 159113>,
 Resource<Res 33483>,
 Resource<Res 515600>,
 Resource<Res 648714>,
 Resource<Res 474294>,
 Resource<Res 

## SQLite resource test

In [13]:
from rtm import SQLiteResourceConnector

In [20]:
manager2w = Manager()

In [21]:
sql_connector = SQLiteResourceConnector(manager2w, "../storage/test.db")

In [22]:
for i in tqdm(range(100_000)):
    n = max(int(np.random.normal(loc=3, scale=2.5)), 1)
    res_tags = random.choices(tags, k=n)
    res = sql_connector.new_resource(f"Res {i+1}", ";;".join(res_tags))
    manager2w.add_resource(res)

100%|██████████| 100000/100000 [06:16<00:00, 265.83it/s]


In [23]:
sql_connector.close()

In [24]:
manager2r = Manager()

In [25]:
sql_connector = SQLiteResourceConnector(manager2r, db_path="../storage/test.db")

In [26]:
sql_connector.load_resources()

In [27]:
manager2r.rename_tag('build:disappoint', 'build:luck')

In [28]:
manager2r.filter_resources(['build:bottle'])

[SQLiteResource<Res 41731 @ 41731>,
 SQLiteResource<Res 41733 @ 41733>,
 SQLiteResource<Res 38674 @ 38674>,
 SQLiteResource<Res 68086 @ 68086>,
 SQLiteResource<Res 41736 @ 41736>,
 SQLiteResource<Res 34597 @ 34597>,
 SQLiteResource<Res 36637 @ 36637>,
 SQLiteResource<Res 67069 @ 67069>,
 SQLiteResource<Res 34600 @ 34600>,
 SQLiteResource<Res 36640 @ 36640>,
 SQLiteResource<Res 62990 @ 62990>,
 SQLiteResource<Res 68093 @ 68093>,
 SQLiteResource<Res 62994 @ 62994>,
 SQLiteResource<Res 12335 @ 12335>,
 SQLiteResource<Res 66056 @ 66056>,
 SQLiteResource<Res 38687 @ 38687>,
 SQLiteResource<Res 68097 @ 68097>,
 SQLiteResource<Res 36648 @ 36648>,
 SQLiteResource<Res 66058 @ 66058>,
 SQLiteResource<Res 62999 @ 62999>,
 SQLiteResource<Res 66059 @ 66059>,
 SQLiteResource<Res 38689 @ 38689>,
 SQLiteResource<Res 63000 @ 63000>,
 SQLiteResource<Res 9281 @ 9281>,
 SQLiteResource<Res 34615 @ 34615>,
 SQLiteResource<Res 14386 @ 14386>,
 SQLiteResource<Res 12347 @ 12347>,
 SQLiteResource<Res 14387 @ 14

In [29]:
sql_connector.close()

## FileStorage resource test

In [30]:
from rtm import FileResourceConnector

In [34]:
manager3w = Manager()

In [35]:
file_connector = FileResourceConnector(manager3w, "../storage/test/")

In [36]:
for i in tqdm(range(100_000)):
    n = max(int(np.random.normal(loc=3, scale=2.5)), 1)
    res_tags = random.choices(tags, k=n)
    res = file_connector.new_resource(f"Res {i+1}", ";;".join(res_tags))
    manager3w.add_resource(res)

100%|██████████| 100000/100000 [00:06<00:00, 15683.53it/s]


In [37]:
manager3r = Manager()

In [38]:
file_connector = FileResourceConnector(manager3r, "../storage/test/")

In [39]:
file_connector.load_resources()

In [40]:
manager3r.rename_tag('build:disappoint', 'build:luck')

In [41]:
manager3r.filter_resources(['build:bottle'])

[FileResource<Res 59965 @ ec9d023ab9b64c24bacffb8cc3307fc5>,
 FileResource<Res 13122 @ 72e732594f8e405ba4d97b2ba9a63b6c>,
 FileResource<Res 63915 @ ddcd19084e914f69b1e823af7f6d8a63>,
 FileResource<Res 96395 @ 34ddeecbcc604d8c99b8673e915232ef>,
 FileResource<Res 25399 @ 74a711aef6f24efaac8042dabbaa8487>,
 FileResource<Res 44223 @ f4d5abec78da4a49bf21ca8c5bfc6fee>,
 FileResource<Res 16633 @ 4f45ad581af644919da89ad5b7aed823>,
 FileResource<Res 66161 @ f3ca7eb447cb48d89640685300fdae2b>,
 FileResource<Res 77309 @ a0b1a96d4cfb4c7399f5b2754fea7357>,
 FileResource<Res 65103 @ f6b1e5d26c9141ab8a38d8fbf5db61c3>,
 FileResource<Res 46700 @ d473ed87a4c94e889dbe516d9a967ffe>,
 FileResource<Res 66413 @ f0ae1a168c654e49a0f1e069ab0c9c77>,
 FileResource<Res 79405 @ ba6edf89902c4b379bf74901e387b2ac>,
 FileResource<Res 48569 @ aaaa848879b9408fa6bc0b903d4a7916>,
 FileResource<Res 40380 @ 44d11db63b46479ebcad44813225cfc1>,
 FileResource<Res 22514 @ bd729d54d2cb4f3a8d163785a194976e>,
 FileResource<Res 69213 

## Test search speed

## Test update/rename speed